In [61]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime

convertTime = lambda x : datetime.strptime(x, "%Y/%m/%d - %H:%M:%S").strftime("%Y-%m-%dT%H:%M:%SZ")

In [62]:
acquisitionGPS = pd.read_csv('results/acquisitionsGPS/gps1.csv')
acquisitionTEL = pd.read_csv('results/acquisitionsTELEPHONE/CSV/acquisition1.csv')

In [63]:
acquisitionGPS['UTC'] = acquisitionGPS['UTC'].apply(convertTime)
acquisitionGPS['UTC'] = pd.to_datetime(acquisitionGPS['UTC'])
acquisitionTEL['UTC'] = pd.to_datetime(acquisitionTEL['UTC'])

In [64]:
merged_df = pd.merge_asof(acquisitionGPS.sort_values('UTC'), acquisitionTEL.sort_values('UTC'), on='UTC', suffixes=('_1', '_2')).dropna()
merged_df

,Timestamp,UTC,Latitude_1,Longitude_1,Latitude_2,Longitude_2
47,1.741710e+09,2025-03-11 16:18:44+00:00,49.384321,1.068865,49.384212,1.069393
48,1.741710e+09,2025-03-11 16:18:45+00:00,49.384324,1.068865,49.384212,1.069393
49,1.741710e+09,2025-03-11 16:18:46+00:00,49.384329,1.068865,49.384212,1.069393
50,1.741710e+09,2025-03-11 16:18:47+00:00,49.384333,1.068868,49.384232,1.069253
51,1.741710e+09,2025-03-11 16:18:48+00:00,49.384338,1.068870,49.384237,1.069247
...,...,...,...,...,...,...
493,1.741710e+09,2025-03-11 16:26:53+00:00,49.384899,1.068054,49.384380,1.068232
494,1.741710e+09,2025-03-11 16:26:54+00:00,49.384909,1.068036,49.384370,1.068218
495,1.741710e+09,2025-03-11 16:26:55+00:00,49.384922,1.068017,49.384378,1.068212
496,1.741710e+09,2025-03-11 16:26:56+00:00,49.384934,1.068000,49.384358,1.068197


In [67]:
from geopy.distance import geodesic

def haversine_distance(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).meters

# Calcul de la distance entre chaque point des deux jeux de données
merged_df['distance'] = merged_df.apply(lambda row: haversine_distance(
    row['Latitude_1'], row['Longitude_1'], row['Latitude_2'], row['Longitude_2']
), axis=1)

print(merged_df[['UTC', 'distance']].head())  # Voir les écarts
min(merged_df['distance'])

                         UTC   distance
47 2025-03-11 16:18:44+00:00  40.224444
48 2025-03-11 16:18:45+00:00  40.395407
49 2025-03-11 16:18:46+00:00  40.489959
50 2025-03-11 16:18:47+00:00  30.184654
51 2025-03-11 16:18:48+00:00  29.571673


0.4261051152366216